### PROYECTO 2: Modelos de clasificación supervisada para evaluar la calidad de un vino.
**Integrantes:** 
* Jonathan Alonso
* Jheison Torres

**Objetivo**
* El objetivo es predecir la calidad del vino basándose en diferentes características físico-químicas.


                  

In [60]:
# Importamos librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

In [26]:
#cargar dataset
df_red=pd.read_csv('../Datos/winequality-red.csv',delimiter=';') 
df_white=pd.read_csv('../Datos/winequality-white.csv',delimiter=';')
df_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [27]:
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [29]:
#valida informacion DF
print('Dataframe Vino Rojo')
df_red.info()
print('Dataframe Vino Blanco')
df_white.info()

Dataframe Vino Rojo
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
Dataframe Vino Blanco
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data colum

In [30]:
#concatenar DF
df_concat=pd.concat([df_red,df_white])
#visualiza primeros registros
df_concat.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [31]:
#verificar info df concatenado
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6497 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 659.9 KB


In [32]:
#describe DF
df_concat.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [49]:
# Función para codificar la calidad
def calidad(value):
    if value <= 5:
        return 'Baja'
    elif 6 <= value <= 7:
        return 'Media'
    else:
        return 'Alta'
 
# Aplicar la función de codificación 
df_concat['quality_label'] = df_concat['quality'].apply(calidad)
 
# Verificar los primeros valores
df_concat.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_label
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Baja
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Baja
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,Baja
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Media
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Baja


In [55]:
#valida frecuencia
df_concat['quality_label'].value_counts()

quality_label
Media    3915
Baja     2384
Alta      198
Name: count, dtype: int64

In [51]:
#eliminamos variable quality
df_concat=df_concat.drop('quality', axis=1)

In [52]:
#Divide caracteristicas del Dataset
X = df_concat.drop('quality_label', axis=1)
y = df_concat['quality_label']

In [53]:
from sklearn.model_selection import train_test_split

## Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [56]:
#RandomForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Entrenamos el modelo
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)

## Predicción
y_pred = modelo_rf.predict(X_test)

## Evaluación
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alta       0.86      0.32      0.47        37
        Baja       0.74      0.75      0.74       451
       Media       0.83      0.85      0.84       812

    accuracy                           0.80      1300
   macro avg       0.81      0.64      0.69      1300
weighted avg       0.80      0.80      0.80      1300



In [57]:
## Regresón Logistica
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression

modelo_lr = LogisticRegression(random_state=0)
modelo_lr.fit(X_train, y_train)

## Predicción
y_pred = modelo_lr.predict(X_test)

## Evaluación
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        Alta       0.00      0.00      0.00        37
        Baja       0.61      0.58      0.59       451
       Media       0.74      0.79      0.77       812

    accuracy                           0.70      1300
   macro avg       0.45      0.46      0.45      1300
weighted avg       0.67      0.70      0.68      1300



In [58]:
## KNN
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier

modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_knn.fit(X_train, y_train)

## Predicción
y_pred = modelo_knn.predict(X_test)

## Evaluación
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alta       0.22      0.14      0.17        37
        Baja       0.63      0.63      0.63       451
       Media       0.76      0.78      0.77       812

    accuracy                           0.71      1300
   macro avg       0.54      0.51      0.52      1300
weighted avg       0.70      0.71      0.71      1300



In [59]:
## Clasificación con SVM
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.metrics import classification_report

modelo_svc = SVC(kernel='linear', C=1.0, random_state=0)
modelo_svc.fit(X_train, y_train)

## Predicción
y_pred = modelo_svc.predict(X_test)

## Evaluación
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        Alta       0.00      0.00      0.00        37
        Baja       0.62      0.59      0.60       451
       Media       0.74      0.80      0.77       812

    accuracy                           0.70      1300
   macro avg       0.45      0.46      0.46      1300
weighted avg       0.68      0.70      0.69      1300



El modelo de **Random Forest** se destaca como el más efectivo para la clasificación de la calidad del vino, con una precisión general del **80%** y un rendimiento sobresaliente en la clase "Media". Sin embargo, presenta limitación en la identificación de la clase "Alta", reflejada en un bajo recall del 32%, indica que aún existen áreas de mejora.